In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

Load the credit card data in 'cc'. 

In [2]:
os.chdir("C://specdata/fraud_kaggle")
cc = pd.read_csv("creditcard.csv")

Check the column names

In [3]:
cc.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

Since isoforest is an unsupervised learning algorithm, it will only use feature data. 'cc_train' dataframe is created with only feature data. The label 'class' is dropped from the cc_train dataframe.

In [4]:
cc_train= cc.drop('Class', 1)

There are two training parameters in this method: the training parameters are the number of trees to build and subsampling size (Liu and Ting, 2008). The are two hyperparameters which can be tunned to ajudt the algorithm. It must be noted that number of tree converges after a certain point. The default vale is 256 for sub sample size and 100 trees. 

In [5]:
clf = IsolationForest(n_estimators=100, max_samples=200)

In [6]:
#Train the model with the data.
clf.fit(cc_train)

IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
        max_samples=200, n_estimators=100, n_jobs=1, random_state=None,
        verbose=0)

Random and recursive partition of data is carried out, which is represented as a tree (random forest). The end of the tree is reached once the recursive partition of data is finished. It is expected that the distance taken to reach the outlier is far less than that for the normal data. The distance of the path is averaged and normalised to calculate the anomaly score. Anomaly score of 1 is considered as an outlier, values close to 0 is considered normal.

In [7]:
# The Anomaly scores are calclated for each observation and stored in 'scores_pred'
scores_pred = clf.decision_function(cc_train)

In [8]:
#verify the length of scores and number of obersvations.
print(len(scores_pred))

284807


In [9]:
print(len(cc))

284807


In [12]:
# scores_pred is added to the cc dataframe 
cc['scores']= scores_pred

In [13]:
#I oberved an conflict with the name 'class'. Therefore, I have changed the name from class to category
cc= cc.rename(columns={'Class': 'Category'})

In [15]:
# Based on (Liu and Ting, 2008), anomalous observation is scored close to 1 
# and non anamolous observations are scored close to zero. 
# I have written a simple loop that will count the number of observation that has score more than 0.5 and is actually anomalous.
counter =0
for n in range(len(cc)):
    if (cc['Category'][n]== 1 and cc['scores'][n] >=0.5):
        counter= counter+1
print (counter)
    
    

0


Clearly, there is something worg with the compatiblity of the algorithm. Lest investigate how the algorithm has scored each observation. Since we have the actual label, it is easy to compare the isolation score to the label. 

In [16]:
# For convinience, divide the dataframe cc based on two labels. 
avg_count_0 = cc.loc[cc.Category==0]    #Data frame with normal observation
avg_count_1 = cc.loc[cc.Category==1]    #Data frame with anomalous observation

In [17]:
#Plot the combined distribution of the scores 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
%pylab.inline
import plotly.plotly as py
import plotly.graph_objs as go
%pylab.inline

x0 = avg_count_0['scores']
x1 = avg_count_1['scores']

Normal = go.Histogram(
    x=x0,
    opacity=0.75
)
Anomaly = go.Histogram(
    x=x1,
    opacity=0.75
)

data = [Normal, Anomaly]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='control Vs experiment')

ERROR:root:Line magic function `%pylab.inline` not found.
ERROR:root:Line magic function `%pylab.inline` not found.
C:\Users\Amar\Anaconda3\lib\site-packages\plotly\plotly\plotly.py:215: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The above graph shows that the distribustion of the score assigned to anomalous and nonanomlous observations are not even. Most of the normal observations are scored more than +0.05 with a mean centered around 0.1.Whereas, distribution of the anomalous observations are spread from -0.1 to +0.5. Inorder to visualise it properly, oth the histogram are drawn separately.

In [18]:
# Distribution of the normal observation scores
x = avg_count_0['scores']
data = [go.Histogram(x=x)]

py.iplot(data, filename='basic histogram')

C:\Users\Amar\Anaconda3\lib\site-packages\plotly\plotly\plotly.py:215: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [19]:
# Distribution of the anomalous observation score
x = avg_count_1['scores']
data = [go.Histogram(x=x)]

py.iplot(data, filename='basic histogram1')

Summary: 
1. It was observed that isoforest can be used as an unsupervised learning for anomaloy detection. 
2. In this particular case, isoforest is not suitable because it tend to score less than 0.05 for anomolous observation. This is against what is proposed by (Liu and Ting, 2008). 
3. This score can be used to carry out smote analysis. We can eliminate observations with score grater than 0.05 and increase the propostion of minority class, significantly. 
4. I will be soon submitting another kernel to test once class SVM for anomaly detection. 

In [20]:
from sklearn import svm

In [21]:
#avg_count_0 = cc.loc[cc.Category==0]
oneclass = svm.OneClassSVM(kernel='poly', gamma=0.0001, nu=0.75)

In [22]:
oneclass

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.0001, kernel='poly',
      max_iter=-1, nu=0.75, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [23]:
oneclass.fit(avg_count_0.loc[0:28432])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.0001, kernel='poly',
      max_iter=-1, nu=0.75, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [24]:
fraud_pred = oneclass.predict(avg_count_1)

In [25]:
unique, counts = np.unique(fraud_pred, return_counts=True)
print (np.asarray((unique, counts)).T)

[[  -1.   86.]
 [   1.  406.]]


In [26]:
df_split= np.array_split(avg_count_0, 10)

In [27]:
df_split[0].shape

(28432, 32)

In [28]:
#fraud_pred = oneclass.predict(df_split[0])
#unique, counts = np.unique(fraud_pred, return_counts=True)
#print (np.asarray((unique, counts)).T)

In [29]:
false_negative= 0
for i in range(1, len(df_split)):
    fraud_pred = oneclass.predict(df_split[i])
    unique, counts = np.unique(fraud_pred, return_counts=True)
    false_negative = false_negative + counts[0]
FN = false_negative/len(avg_count_0)
print (FN)
    

0.899998241387


In [30]:
false_negative

255883

In [31]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [32]:
svr = svm.OneClassSVM()

In [34]:
from sklearn.model_selection import GridSearchCV
clfgscv = GridSearchCV(svr, parameters)

In [ ]:
X_train= avg_count_0.drop('Class', 1)

In [35]:
clfgscv.fit(avg_count_0, avg_count_1)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False) does not.